In [1]:
import torch
import torch.nn as nn
from sinabs.backend.dynapcnn import DynapcnnNetworkGraph
from sinabs.layers import Merge, IAFSqueeze

In [2]:
torch.manual_seed(0)

In [3]:
channels = 2
height = 34
width = 34

input_shape = (channels, height, width)

## Network Module

We need to define a `nn.Module` implementing the network we want the chip to reproduce.

In [4]:
class SNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(2, 10, 2, 1, bias=False)
        self.conv1_iaf = IAFSqueeze(batch_size=1)
        self.pool1 = nn.AvgPool2d(3,3)
        self.pool1a = nn.AvgPool2d(4,4)

        self.conv2 = nn.Conv2d(10, 10, 4, 1, bias=False)
        self.conv2_iaf = IAFSqueeze(batch_size=1)

        self.conv3 = nn.Conv2d(10, 1, 2, 1, bias=False)
        self.conv3_iaf = IAFSqueeze(batch_size=1)

        self.flat = nn.Flatten()

        self.fc1 = nn.Linear(49, 100, bias=False)
        self.fc1_iaf = IAFSqueeze(batch_size=1)
        
        self.fc2 = nn.Linear(100, 100, bias=False)
        self.fc2_iaf = IAFSqueeze(batch_size=1)

        self.fc3 = nn.Linear(100, 10, bias=False)
        self.fc3_iaf = IAFSqueeze(batch_size=1)

        self.merge1 = Merge()

    def forward(self, x):
        # -- conv. block 1 --
        con1_out = self.conv1(x)
        conv1_iaf_out = self.conv1_iaf(con1_out)
        pool1_out = self.pool1(conv1_iaf_out)
        pool1a_out = self.pool1a(conv1_iaf_out)
        # -- conv. block 2 --
        conv2_out = self.conv2(pool1_out)
        conv2_iaf_out = self.conv2_iaf(conv2_out)
        # -- conv. block 3 --
        merge1_out = self.merge1(pool1a_out, conv2_iaf_out)
        conv3_out = self.conv3(merge1_out)
        conv3_iaf_out = self.conv3_iaf(conv3_out)
        flat_out = self.flat(conv3_iaf_out)
        # -- fc clock 1 --
        fc1_out = self.fc1(flat_out)
        fc1_iaf_out = self.fc1_iaf(fc1_out)
        # -- fc clock 2 --
        fc2_out = self.fc2(fc1_iaf_out)
        fc2_iaf_out = self.fc2_iaf(fc2_out)
        # -- fc clock 3 --
        fc3_out = self.fc3(fc2_iaf_out)
        fc3_iaf_out = self.fc3_iaf(fc3_out)

        return fc3_iaf_out

In [5]:
snn = SNN()

The following exemplifies how each of the layers in the SNN should be grouped together to form a `DynapcnnLayer` instance. Let's check the shapes of the tensors each of the (original) layers in the model inputs and outputs by feeding a fake input through the model:

In [6]:
x = torch.randn((1, *input_shape))

# -- conv. block 1 --
con1_out = snn.conv1(x)
print(con1_out.shape)
conv1_iaf_out = snn.conv1_iaf(con1_out)
print(conv1_iaf_out.shape)
pool1_out = snn.pool1(conv1_iaf_out)
print(pool1_out.shape)
pool1a_out = snn.pool1a(conv1_iaf_out)
print(pool1a_out.shape)
# -- conv. block 2 --
conv2_out = snn.conv2(pool1_out)
print(conv2_out.shape)
conv2_iaf_out = snn.conv2_iaf(conv2_out)
print(conv2_iaf_out.shape)
# -- conv. block 3 --
merge1_out = snn.merge1(pool1a_out, conv2_iaf_out)
print(merge1_out.shape)
conv3_out = snn.conv3(merge1_out)
print(conv3_out.shape)
conv3_iaf_out = snn.conv3_iaf(conv3_out)
print(conv3_iaf_out.shape)
flat_out = snn.flat(conv3_iaf_out)
print(flat_out.shape)
# -- fc clock 1 --
fc1_out = snn.fc1(flat_out)
print(fc1_out.shape)
fc1_iaf_out = snn.fc1_iaf(fc1_out)
print(fc1_iaf_out.shape)
# -- fc clock 2 --
fc2_out = snn.fc2(fc1_iaf_out)
print(fc2_out.shape)
fc2_iaf_out = snn.fc2_iaf(fc2_out)
print(fc2_iaf_out.shape)
# -- fc clock 3 --
fc3_out = snn.fc3(fc2_iaf_out)
print(fc3_out.shape)
fc3_iaf_out = snn.fc3_iaf(fc3_out)
print(fc3_iaf_out.shape)

torch.Size([1, 10, 33, 33])
torch.Size([1, 10, 33, 33])
torch.Size([1, 10, 11, 11])
torch.Size([1, 10, 8, 8])
torch.Size([1, 10, 8, 8])
torch.Size([1, 10, 8, 8])
torch.Size([1, 10, 8, 8])
torch.Size([1, 1, 7, 7])
torch.Size([1, 1, 7, 7])
torch.Size([1, 49])
torch.Size([1, 100])
torch.Size([1, 100])
torch.Size([1, 100])
torch.Size([1, 100])
torch.Size([1, 10])
torch.Size([1, 10])


## DynapcnnNetwork Class

In the constructor of `DynapcnnNetworkGraph` the SNN passed as argument (defined as a `nn.Module`) will be parsed such that each layer is represented in a computational graph (using `nirtorch.extract_torch_graph`). 

The layers are the `nodes` of the graph, while their connectivity (how the outputs from a layer are sent to other layers) is represented as `edges`, represented in a `list` of `tuples`.

Once the constructor finishes its initialization, the `hw_model.dynapcnn_layers` property is a dictionary where each entry represents the ID of a `DynapcnnLayer` instance (an `int` from `0` to `L`), with this entry containing a `DynapcnnLayer` instance where a subset of the layers in the original SNN has been incorporated into, the core such instance has been assigned to, and the list of `DynapcnnLayer` instances (their IDs) the layer targets.

In [7]:
hw_model = DynapcnnNetworkGraph(
    snn,
    discretize=True,
    input_shape=input_shape
)

The `hw_model.to()` call will figure out into which core eac `DynapcnnLayer` instance will be assigned to. Once this assingment is made the instance itself is used to configure the `CNNLayerConfig` instance representing the core's configuration.

If the cores' configuration is valid, each `DynapcnnLayer` instance and their respective destinations will be used to create a computational graph that encodes how the `forward` method of `hw_model.network` (a `nn.Module` using the `DynapcnnLayer` instances) propagates that through the network.

In [8]:
hw_model.to(device="speck2fmodule:0")

Network is valid
(0, 1)
(0, 2)
(1, 2)
(2, 3)
(3, 4)
(4, 5)


RuntimeError: Device is already opened!

The layers comprising our `hw_model` and their respective metadata can be inspected by calling `print` on a `DynapcnnNetworkGraph` instance.

In [ ]:
print(hw_model)

---- DynapcnnLayer 0 ----------------------------------------------------------
> layer modules: 
(node 0): Conv2d(2, 10, kernel_size=(2, 2), stride=(1, 1), bias=False)
(node 1): IAFSqueeze(spike_threshold=Parameter containing:
tensor(1.), min_v_mem=Parameter containing:
tensor(-32768.), batch_size=1, num_timesteps=-1)
(node 2): SumPool2d(norm_type=1, kernel_size=(3, 3), stride=None, ceil_mode=False)
(node 3): SumPool2d(norm_type=1, kernel_size=(4, 4), stride=None, ceil_mode=False)
> layer destinations: [1, 2]
> assigned core: 0

---- DynapcnnLayer 1 ----------------------------------------------------------
> layer modules: 
(node 4): Conv2d(10, 10, kernel_size=(4, 4), stride=(1, 1), bias=False)
(node 6): IAFSqueeze(spike_threshold=Parameter containing:
tensor(1.), min_v_mem=Parameter containing:
tensor(-32768.), batch_size=1, num_timesteps=-1)
> layer destinations: [2]
> assigned core: 1

---- DynapcnnLayer 2 ----------------------------------------------------------
> layer modules: